In [20]:
import os
import json

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

In [22]:
# Instancia de modelo a reusarse
from framework_genai.GenAIMemory import GenAIMemory, EnumMemoryType
from framework_genai.GenAILLM import GenAILLM, EnumGenAIPlatforms, EnumGenAIModelsIdsOpenAI
from framework_genai.LoggingAndTelemetry import EnumLogs
import os

ai_prefix = "Human"
human_prefix = "Assistant"

gen_ai_llm = GenAILLM(
    platform = EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id = EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO,
    parameters_inference = {
        'max_tokens': 512, 
        "temperature": 0,
        "top_p": 0.9
    },
    ai_prefix = ai_prefix,
    human_prefix = human_prefix,
    component_memory=GenAIMemory(EnumMemoryType.MEMORY_CHAT_BUFFER, ai_prefix, human_prefix),
    platform_configuration = {
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]
    },
    verify_ssl=True,
    verbose_level=EnumLogs.LOG_LEVEL_DEBUG
)
print(gen_ai_llm.llm_model.verbose)
print(gen_ai_llm.invoke("Hola", return_completion_only=True, save_in_memory=False))

2024-02-06 15:40:40.694069-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO construido
True
2024-02-06 15:40:40.694069-05:00: Invocando modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO con Hola
¡Hola! ¿En qué puedo ayudarte hoy?


In [23]:
from framework_genai.GenAIOutputParser import GenAIOutputParser
gen_ai_output_parser = GenAIOutputParser([
    {
        "name": "reasoning",
        "description": "razonamiento detras de la respuesta"
    },
    {
        "name": "respuesta",
        "description": "respuesta final concreta y concisa"
    },
])

In [29]:
from framework_genai.GenAIPrompt import EnumPromptsBase
print(EnumPromptsBase.PROMPT_BASE_REACT_CHAT.value)

{system_prompt}. 
    Tienes acceso a las siguientes herramientas:
    {tools}

    Este es un contexto adicional: <context>{context}</context>

    Utilice el siguiente formato puesto en la etiqueta <formato> para generar tu respuesta final:
    <formato>
    Question: la pregunta de entrada que debes responder
    Thought: siempre debes pensar qué hacer
    Action: la acción a tomar, debe ser una de [{tool_names}]
    Action Input: la entrada de la acción
    Observation: el resultado de la acción
    ... (la secuencia Thought,Action,Action Input y Observation puede repetirse tantas veces como sea necesario)
    Thought: Ahora sé la respuesta final
    Final Answer: la respuesta final a la pregunta de entrada original, es un string que estará en el formato de las siguientes instrucciones: {format_instructions}
    </formato>

    Ahora inicia. Te pasaré un Question del que deberás continuar con la cadena de pensamientos. Recuerda utilizar saltos de línea para denotar cada Thought/Act

In [24]:
from framework_genai.GenAIAgent import GenAIAgent, EnumGenAIAgentType
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
import random
from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

@tool("lower_case", return_direct=False)
def to_lower_case(input:str) -> str:
    """Returns the input as all lower case."""
    return input.lower()

@tool("random_number", return_direct=False)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    return random.randint(0, 100)

python_repl_tool = PythonREPLTool()
search = GoogleSerperAPIWrapper()
tools = [
    to_lower_case,
    random_number_maker,
    python_repl_tool,
    Tool(
        name="Buscador",
        func=search.run,
        description="útil cuando deseamos buscar algo"
    )
]

agent_react = GenAIAgent(
    system_prompt="Eres un asistente muy util",
    agent_type=EnumGenAIAgentType.AGENT_TYPE_REACT,
    gen_ai_output_parser=gen_ai_output_parser,
    gen_ai_llm=gen_ai_llm,
    tools=tools,
    state={},
    name="AgenteDemo",
    verbose_level=EnumLogs.LOG_LEVEL_DEBUG
)
print(agent_react.agent_executor.verbose)
agent_result = agent_react.invoke({
    "input": "¿Quién es el presidente de ecuador? ¿Cuánto es su edad dividido para 2?",
    "context": ""
})
print(agent_result)

True


> Entering new AgentExecutor chain...
Necesito buscar quién es el presidente de Ecuador y su edad actual. Puedo usar la herramienta de Buscador para encontrar esta información. 
Action: Buscador
Action Input: "presidente de Ecuador"Daniel NoboaAhora que tengo el nombre del presidente de Ecuador, puedo buscar su edad actual. Utilizaré la herramienta de Buscador nuevamente para encontrar esta información.
Action: Buscador
Action Input: "edad de Daniel Noboa"30 de noviembre de 1987 (edad 36 años), Miami, Florida

Python REPL can execute arbitrary code. Use with caution.


Ahora que tengo la fecha de nacimiento de Daniel Noboa, puedo calcular su edad actual dividiendo el año actual menos el año de nacimiento entre 2. Utilizaré la herramienta de Python_REPL para realizar este cálculo.

Action: Python_REPL
Action Input: print((2023 - 1987) / 2)18.0
Ahora sé la respuesta final. El presidente de Ecuador es Daniel Noboa y su edad dividida por 2 es 18 años.

Final Answer:
```json
{
	"reasoning": "Utilicé la herramienta de Buscador para encontrar el nombre del presidente de Ecuador, Daniel Noboa. Luego, utilicé la misma herramienta para encontrar su fecha de nacimiento y calcular su edad actual dividiendo el año actual menos el año de nacimiento entre 2.",
	"respuesta": "El presidente de Ecuador es Daniel Noboa y su edad dividida por 2 es 18 años."
}
```

> Finished chain.
{'input': '¿Quién es el presidente de ecuador? ¿Cuánto es su edad dividido para 2?', 'context': '', 'output': '```json\n{\n\t"reasoning": "Utilicé la herramienta de Buscador para encontrar el